In [ ]:
from __future__ import print_function

from keras.engine import Model
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense

import numpy as np

from pre_processing import load_data

In [ ]:
# Load the data.
(x_train, y_train), (x_test, y_test), vocab = load_data(num_words=0, train_fraction=0.7, speaker='s1')
print('Number of words for training: ', x_train.shape[0])
print('Number of words for testing: ', x_test.shape[0])

In [ ]:
# Build the LSTM model.
model = Sequential()
model.add(LSTM(128, input_shape=x_train[0].shape, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(vocab), activation='tanh'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model.
model.fit(x_train, y_train, batch_size=32, epochs=20, verbose=1)

In [ ]:
# Test the model.
score, acc = model.evaluate(x_test, y_test, batch_size=32)
print('Test score: ', score)
print('Test accuracy: ', acc)

In [ ]:
# Try matching some results.
i = 4
a = model.predict(x_test[i].reshape(1, 6, 50))
best = np.argsort(a.reshape(len(vocab)))[::-1][0]

des_index = np.argsort(y_test[i])[::-1][0]
act_index = best

for key, val in vocab.iteritems():
    if val == des_index:
        des_word = key
    if val == act_index:
        act_word = key

print(des_word)
print(act_word)